In [8]:
!pip install autodistill==0.1.29 supervision==0.27.0 opencv-python roboflow
!pip install autodistill-grounded-sam-2 --no-deps

In [12]:
import huggingface_hub
from huggingface_hub import hf_hub_download
import functools
from tqdm import tqdm

# Enable verbose logs
huggingface_hub.logging.set_verbosity_info()

# Store original function
orig_download = hf_hub_download

# Wrap hf_hub_download with tqdm print
@functools.wraps(orig_download)
def patched_hf_download(*args, **kwargs):
    repo_id = args[0] if len(args) > 0 else "repo"
    filename = kwargs.get("filename", None)
    if filename:
        print(f"📥 Downloading: {repo_id}/{filename}")
    else:
        print(f"📥 Downloading from: {repo_id}")
    return orig_download(*args, **kwargs)

# Apply patch
huggingface_hub.hf_hub_download = patched_hf_download

print("🟢 Progress-bar logging enabled! 🚀")


🟢 Progress-bar logging enabled! 🚀


In [ ]:
from autodistill_grounded_sam_2 import GroundedSAM2
from autodistill.detection import CaptionOntology
import supervision as sv
import cv2, os

model = GroundedSAM2(
    ontology=CaptionOntology({"white car": "car"}),
    model="Grounding DINO (SwinT)",
    grounding_dino_box_threshold=0.25
)


print("🚗 Model Loaded!")


In [ ]:
from google.colab import files
uploaded = files.upload()
image_paths = list(uploaded.keys())
image_paths

In [ ]:
os.makedirs("results", exist_ok=True)

for img in image_paths:
    result = model.predict(img).with_nms()

    mask_annotator = sv.MaskAnnotator()

    annotated = mask_annotator.annotate(
        cv2.imread(img),
        detections=result
    )


    save_path = f"results/{os.path.splitext(img)[0]}.png"
    cv2.imwrite(save_path, annotated)
    print("💾 Saved:", save_path)

print("🎉 Done — saved PNGs in /results!")
